# 卷积神经网络
## 全连接层-卷积
### 不变性
- **平移不变性**：不管检测对象出现在图像中的哪个位置，神经网络的前面几层
应该对相同的图像区域具有相似的反应，即为“平移不变性”。
- **局部性**：神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远区域的关系，这就是“局部性”原则。
### 多层感知机的限制
- 参数量巨大。
- 空间信息丢失（因为输入模型前会强行拉伸为一个一维向量）。
- 输入尺寸固定。
### 卷积
- 卷积核（矩阵）滑动进行一个点乘求和的过程，最终得到一个新的矩阵（张量，特征图）。
## 图像卷积
- 说是卷积其实也就是一个互相关运算。


In [73]:
import numpy as np
import torch


def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

"""测试代码"""
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
result = corr2d(X, K)
print(result)

tensor([[19., 25.],
        [37., 43.]])


### 卷积层构造

In [74]:
import torch.nn as nn

class Conv2D(nn.Module):
    def __init__(self, kernel_size, **kwargs):
        super().__init__(**kwargs)
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        return self.corr2d(x, self.weight) + self.bias
    
    @staticmethod
    def corr2d(X, K):
        h, w = K.shape
        Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
        for i in range(Y.shape[0]):
            for j in range(Y.shape[1]):
                Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
        return Y

### 图像中的目标边缘检测
- 黑白边缘检测（只需要一维简单的卷积核）

In [75]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [76]:
K = np.array([[1, -1]])
K

array([[ 1, -1]])

In [77]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [78]:
corr2d(torch.transpose(X, 0, 1), K)  # 转置

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

- 二维卷积层

In [79]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1, 1, 6, 7)

lr = 3e-2

optimizer = torch.optim.SGD(conv2d.parameters(), lr=lr)

for i in range(10):
    """前向传播"""
    Y_hat= conv2d(X)
    l = (Y_hat - Y) ** 2
    
    """反向传播"""
    conv2d.zero_grad()
    l.sum().backward()
    
    # """手动更新权重"""
    # with torch.no_grad():
    #     conv2d.weight.data -= lr * conv2d.weight.grad
    
    optimizer.step()    
    
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum().detach().item():.3f}')

epoch 2, loss 8.847
epoch 4, loss 1.507
epoch 6, loss 0.262
epoch 8, loss 0.048
epoch 10, loss 0.010


In [80]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9899, -0.9793]])

**注**：在 $\mathrm{CNN}$ 中卷积运算和互相关运算的效果差不多，但后者更方便，但是在数学中的含义是不同的。
### 特征映射和感受野
- 特征映射就是输出的卷积层。
- 感受野就是特征矩阵每次被卷积核覆盖的那一部分。
## 填充和步幅
### 填充
- 就是为了保持特征图（空间）的维度（可以联系放大缩小图片来简单理解一下）。
- 保护边界信息（边缘信息在没有填充的情况下参与计算的次数比其他位置的信息少）。

In [81]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=False)
X = torch.from_numpy(np.random.uniform(size=(8, 8))).float()
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [82]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 步幅
- 就是卷积核一次移动的行、列数（分别对应水平、垂直步幅）。

In [83]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [84]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

## 多输入多通道
### 多输入通道

In [85]:
def corr2d_multi_in_pytorch(X, K):
    X = X.unsqueeze(0)
    K = K.unsqueeze(0)
    
    output = torch.nn.functional.conv2d(X, K)
    
    return output.squeeze(0)

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]], [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in_pytorch(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]]])

### 多输出通道

In [86]:
def corr2d_multi_out_pytorch(X, K):
    """使用PyTorch内置函数实现多输入多输出通道互相关"""
    X = X.unsqueeze(0)
    output = torch.nn.functional.conv2d(X, K)
    return output.squeeze(0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [87]:
corr2d_multi_out_pytorch(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 1 $\times$ 1卷积层

In [88]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    
    Y = torch.mm(K, X)  # 使用torch.mm进行矩阵乘法
    return Y.reshape((c_o, h, w))

In [89]:
X = torch.randn(3, 3, 3)
K = torch.randn(2, 3, 1, 1)

In [90]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_out_pytorch(X, K)
assert float(np.abs(Y1 - Y2).sum()) < 1e-6
print(Y1,"\n",Y2)

tensor([[[ 1.9549,  0.5617, -1.0108],
         [ 0.7516, -0.3996, -1.0445],
         [ 0.9544, -1.9995, -2.2511]],

        [[ 2.7576,  2.0785, -1.1758],
         [ 0.9664, -1.1479, -0.3568],
         [ 0.7093, -3.3872, -2.6560]]]) 
 tensor([[[ 1.9549,  0.5617, -1.0108],
         [ 0.7516, -0.3996, -1.0445],
         [ 0.9544, -1.9995, -2.2511]],

        [[ 2.7576,  2.0785, -1.1758],
         [ 0.9664, -1.1479, -0.3568],
         [ 0.7093, -3.3872, -2.6560]]])


## 汇聚（池化）层
- 对卷积层输出的特征图进行降维压缩，从而提炼和浓缩信息（就是划一个窗口里面计算选一个值来代替）。
### 最大汇聚（池化）层
- 选择窗口中的最大值。
### 平均汇聚（池化）层
- 计算窗口中的平均值。

In [91]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

X = np.array([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [92]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

### 填充和步幅
- 和卷积里面的含义差不多。

In [93]:
X = np.arange(16, dtype=np.float32).reshape(1, 1, 4, 4)
X

array([[[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]]]], dtype=float32)

In [94]:
pool2d = nn.MaxPool2d(3)
pool2d(torch.from_numpy(X))

tensor([[[[10.]]]])

In [95]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(torch.from_numpy(X))

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [96]:
pool2d = nn.MaxPool2d((2, 3), padding=(0, 1), stride=(2, 3))
pool2d(torch.from_numpy(X))

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

### 多个通道
- 也是类似卷积里面的多通道。

In [97]:
X = np.concatenate((X, X + 1), 1)
X

array([[[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]],

        [[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.],
         [ 9., 10., 11., 12.],
         [13., 14., 15., 16.]]]], dtype=float32)

In [98]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(torch.from_numpy(X))

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

## 卷积神经网络
![卷积神经网络](../image/ConvolutionalNeuralNetworks.jpg)

In [99]:
class LaNet(nn.Module):
    def __init__(self):
        super(LaNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
        
        # 先进行一次前向传播来计算特征数
        with torch.no_grad():
            dummy_input = torch.randn(1, 1, 28, 28)
            dummy_output = self.features(dummy_input)
            self.num_features = dummy_output.numel() // dummy_output.shape[0]  # 除batch_size
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.num_features, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x
    
net = LaNet()

print(net)

LaNet(
  (features): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): Sigmoid()
    (3): Linear(in_features=120, out_features=84, bias=True)
    (4): Sigmoid()
    (5): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [100]:
X = torch.rand(size=(1, 1, 28, 28))

for name, layer in net.named_children():
    if name == 'features' or name == 'classifier':
        print(f"\n==={name}===")
        for sub_name, sub_layer in layer.named_children():
            X = sub_layer(X)
            print(f'{sub_name}: {type(sub_layer).__name__} output shape:\t{X.shape}')   
    else:
        X = layer(X)
        print(f'{name}: {type(layer).__name__} output shape:\t{X.shape}')


===features===
0: Conv2d output shape:	torch.Size([1, 6, 28, 28])
1: Sigmoid output shape:	torch.Size([1, 6, 28, 28])
2: AvgPool2d output shape:	torch.Size([1, 6, 14, 14])
3: Conv2d output shape:	torch.Size([1, 16, 10, 10])
4: Sigmoid output shape:	torch.Size([1, 16, 10, 10])
5: AvgPool2d output shape:	torch.Size([1, 16, 5, 5])

===classifier===
0: Flatten output shape:	torch.Size([1, 400])
1: Linear output shape:	torch.Size([1, 120])
2: Sigmoid output shape:	torch.Size([1, 120])
3: Linear output shape:	torch.Size([1, 84])
4: Sigmoid output shape:	torch.Size([1, 84])
5: Linear output shape:	torch.Size([1, 10])


In [101]:
"""使用Sequential构建网络"""
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2),
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5),
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120),
    nn.Sigmoid(),
    nn.Linear(120, 84),
    nn.Sigmoid(),
    nn.Linear(84, 10)
)

"""生成随机输入"""
X = torch.rand(size=(1, 1, 28, 28))
print("输入形状:", X.shape)

"""遍历每一层并打印输出形状"""
for i, layer in enumerate(net):
    X = layer(X)
    print(f'Layer {i}: {layer.__class__.__name__} output shape:\t{X.shape}')

输入形状: torch.Size([1, 1, 28, 28])
Layer 0: Conv2d output shape:	torch.Size([1, 6, 28, 28])
Layer 1: Sigmoid output shape:	torch.Size([1, 6, 28, 28])
Layer 2: AvgPool2d output shape:	torch.Size([1, 6, 14, 14])
Layer 3: Conv2d output shape:	torch.Size([1, 16, 10, 10])
Layer 4: Sigmoid output shape:	torch.Size([1, 16, 10, 10])
Layer 5: AvgPool2d output shape:	torch.Size([1, 16, 5, 5])
Layer 6: Flatten output shape:	torch.Size([1, 400])
Layer 7: Linear output shape:	torch.Size([1, 120])
Layer 8: Sigmoid output shape:	torch.Size([1, 120])
Layer 9: Linear output shape:	torch.Size([1, 84])
Layer 10: Sigmoid output shape:	torch.Size([1, 84])
Layer 11: Linear output shape:	torch.Size([1, 10])


In [102]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

batch_size = 256

transform = transforms.Compose([
    transforms.ToTensor(),  # 将PIL图像转换为Tensor，并归一化到[0,1]
])

# 加载训练集和测试集
train_dataset = datasets.FashionMNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)

test_dataset = datasets.FashionMNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform
)

# 创建数据加载器
train_iter = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True,  # 训练集需要打乱
    num_workers=2  # 使用多进程加载数据
)

test_iter = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False,  # 测试集不需要打乱
    num_workers=2
)

100%|██████████| 26.4M/26.4M [19:35<00:00, 22.5kB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 40.4kB/s]
100%|██████████| 4.42M/4.42M [02:45<00:00, 26.7kB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 5.15MB/s]


In [103]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    if device is None:
        device = next(net.parameters().device if list(net.parameters()) else torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
    correct = 0
    total = 0
    net.eval()
    
    with torch.no_grad():
        for X, y in data_iter:
            X = X.to(device)
            y = y.to(device)
            
            outputs = net(X)
            _, predicted = torch.max(outputs.data, 1)
            
            total += y.size(0)
            correct += (predicted == y).sum().item()
            
    return correct / total

In [104]:
import time

class Timer:
    """简单的计时器类"""
    def __init__(self):
        self.times = []
        self.start()
    
    def start(self):
        """启动计时器"""
        self.tik = time.time()
    
    def stop(self):
        """停止计时器并记录时间"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]
    
    def avg(self):
        """返回平均时间"""
        return sum(self.times) / len(self.times)
    
    def sum(self):
        """返回时间总和"""
        return sum(self.times)
    
    def cumsum(self):
        """返回累计时间"""
        return torch.tensor(self.times).cumsum(dim=0).tolist()

class Accumulator:
    """用于累加多个变量的类"""
    def __init__(self, n):
        self.data = [0.0] * n
    
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    
    def reset(self):
        self.data = [0.0] * len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def accuracy(y_hat, y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

def train_cnn_t(net, train_iter, test_iter, num_epochs, lr, device):
    """GPU训练"""
    def init_weights(m):
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
                
    net.apply(init_weights)
    net.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    
    timer = Timer()
    num_batches = len(train_iter)
    
    train_losses = []
    train_accs = []
    test_accs = []
    
    for epoch in range(num_epochs):
        metric = Accumulator(3)
        net.train()
        
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            X, y = X.to(device), y.to(device)
            
            optimizer.zero_grad()
            y_hat = net(X)
            loss = criterion(y_hat, y)
            
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                acc = accuracy(y_hat, y)
                metric.add(loss.item() * X.shape[0], acc, X.shape[0])
                
            timer.stop()
            
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                train_l = metric[0] / metric[2]
                train_acc = metric[1] / metric[2]
                print(f'Epoch [{epoch+1}/{num_epochs}],'
                      f'Step [{i+1}/{num_batches}],'
                      f'Loss: {train_l:.3f},'
                      f'Acc: {train_acc:.3f}')
                
        test_acc = evaluate_accuracy_gpu(net, test_iter, device)
        train_l = metric[0] / metric[2]
        train_acc = metric[1] / metric[2]
        
        train_losses.append(train_l)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_l:.3f}, '
              f'Train Acc: {train_acc:.3f}, '
              f'Test Acc: {test_acc:.3f}')
        
    """计算训练速度"""
    total_examples = metric[2] * num_epochs
    total_time = timer.sum()
    speed = total_examples / total_time
    
    print(f'{speed:.1f} examples/sec on {device}')
    
    return train_losses, train_accs, test_accs

In [105]:
def try_gpu(i=0):
    """如果存在GPU，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

# 设置超参数
lr, num_epochs = 0.9, 10

# 调用训练函数
train_cnn_t(net, train_iter, test_iter, num_epochs, lr, try_gpu())

Epoch [1/10],Step [47/235],Loss: 2.372,Acc: 0.100
Epoch [1/10],Step [94/235],Loss: 2.343,Acc: 0.099
Epoch [1/10],Step [141/235],Loss: 2.331,Acc: 0.103
Epoch [1/10],Step [188/235],Loss: 2.322,Acc: 0.105
Epoch [1/10],Step [235/235],Loss: 2.273,Acc: 0.130
Epoch [1/10], Train Loss: 2.273, Train Acc: 0.130, Test Acc: 0.306
Epoch [2/10],Step [47/235],Loss: 1.448,Acc: 0.427
Epoch [2/10],Step [94/235],Loss: 1.313,Acc: 0.473
Epoch [2/10],Step [141/235],Loss: 1.225,Acc: 0.508
Epoch [2/10],Step [188/235],Loss: 1.161,Acc: 0.532
Epoch [2/10],Step [235/235],Loss: 1.115,Acc: 0.550
Epoch [2/10], Train Loss: 1.115, Train Acc: 0.550, Test Acc: 0.606
Epoch [3/10],Step [47/235],Loss: 0.894,Acc: 0.644
Epoch [3/10],Step [94/235],Loss: 0.878,Acc: 0.652
Epoch [3/10],Step [141/235],Loss: 0.863,Acc: 0.658
Epoch [3/10],Step [188/235],Loss: 0.841,Acc: 0.669
Epoch [3/10],Step [235/235],Loss: 0.828,Acc: 0.674
Epoch [3/10], Train Loss: 0.828, Train Acc: 0.674, Test Acc: 0.683
Epoch [4/10],Step [47/235],Loss: 0.722,A

([2.2733701802571615,
  1.115173760732015,
  0.828308590221405,
  0.705748969300588,
  0.6375064541180928,
  0.5908440706888834,
  0.5505581643422445,
  0.517240756225586,
  0.4893931660016378,
  0.4670578139781952],
 [0.13031666666666666,
  0.5502833333333333,
  0.6736166666666666,
  0.7230333333333333,
  0.7521333333333333,
  0.7704833333333333,
  0.78745,
  0.8019,
  0.8149166666666666,
  0.8240166666666666],
 [0.3064,
  0.6059,
  0.6829,
  0.7259,
  0.7489,
  0.7546,
  0.7815,
  0.764,
  0.7909,
  0.8094])